# Cookbook gallery
This chapter is simply for displaying beautiful and creative plots made using toytree and toyplot. If you have one of your own please reach out (or make a github pull request) to contribute it. You can use simulated data (see examples below) or show examples with real data. For simulated data please limit data generation to the use of numpy and pandas. If real data, please make sure that the trees you use are available in an archived location (reliable URL) so that the plot can be easily re-created. 

In [1]:
import numpy as np
import toytree
import toyplot

### 1. ToyTree + barplot
Aligning a tree with data is sometimes easier on one axis versus two. See the (#1) and (#2) for comparison. Here when plotting on one axis the tree coordinates which map to treeheight and the number of tips can be difficult to align with data (e.g., a barplot) since the data values may be much greater than the treeheight. This can be fixed by tranforming the data and the axis labels. The example on a two axes is a bit easier in this case.  

In [2]:
# generate a random tree and data
ntips = 20
rseed = 123456
np.random.seed(rseed)
rtre = toytree.rtree.unittree(ntips=ntips, seed=rseed)
randomdata = np.random.uniform(20, 200, ntips)

# setup a toyplot Canvas and Cartesian grid with two axes
canvas = toyplot.Canvas(width=375, height=350)
ax0 = canvas.cartesian(bounds=(50, 225, 50, 300))
ax1 = canvas.cartesian(bounds=(190, 325, 50, 300))

# add tree to canvas
rtre.draw(axes=ax0);
ax0.show = False

# plot the data 
# (y-axis is range 0-ntips);
# (x-axis is bar values transformed to be 0-1)
# baseline is the space between tipnames and bars
ax1.bars(
    np.arange(ntips), 
    randomdata,
    along='y',
);
ax1.show = True
ax1.y.show = False
ax1.x.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="350.0px" id="t9673f1165ca74183b9d93fa89357328f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 375.0 350.0" width="375.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r14 r16 r19 r17 r5 r8 r18 r12 r0 r1 r10 r9 r13 r11 r7 r2 r4 r15 r3 r6 0 50 100 150 200

### 2. Spacing tree vs. tip names

You can see that in both cases the treeheight is between 0 and -1 on the x-axis. Because the tip labels get really long in this plot, they end up getting cutoff on the right side. This can be fixed either by making the whole canvas wider, or, by extending the domain of the axis beyond where the data is located (the tree is treated as data whereas text is not). I show the x-axis in this case to highlight where the data are located on the x, and where the domain is by default and when extended. 

In [3]:
# generate a random tree and data
ntips = 20
rseed = 123456
rtre = toytree.rtree.unittree(ntips=ntips, seed=rseed)

# get a list of long names so this is a better example
names = ["".join(np.random.choice(list("abcd"), i + 1)) for i in range(ntips)]

# make a canvas and coords for two plots
canvas = toyplot.Canvas(width=600, height=350)
ax0 = canvas.cartesian(grid=(1, 2, 0))
ax1 = canvas.cartesian(grid=(1, 2, 1))
ax0.y.show = False
ax1.y.show = False

# plot the tree with its default spacing for tree and names
rtre.draw(tip_labels=names, axes=ax0);

# plot the tree on the second axis
rtre.draw(tip_labels=names, axes=ax1);

# extend the domain of the axis on the second plot
ax1.x.domain.max = 3.0

<svg class="toyplot-canvas-Canvas" height="350.0px" id="tb9e8055c11c94b53ac2c4e42fb0db975" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 350.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> b bd add bdaa ccdda dacbcc aacaaba cddddacb cabacabcb adacacbcdc dbaadabbaac cdbbbadadaaa aaabdcdbbaaca dcdbbccdcccdba bdccbdcdaababba cdadadabbdabacdb acaccbaddbbabdbbb caddbdcbaccbdaccdc daabdcaacbcdbdabcdb cabcdddaddcccccbadcb -1.0 -0.5 0.0 0.5 1.0 b bd add bdaa ccdda dacbcc aacaaba cddddacb cabacabcb adacacbcdc dbaadabbaac cdbbbadadaaa aaabdcdbbaaca dcdbbccdcccdba bdccbdcdaababba cdadadabbdabacdb acaccbaddbbabdbbb caddbdcbaccbdaccdc daabdcaacbcdbdabcdb cabcdddaddcccccbadcb -1 0 1 2 3

### 3. Node size/color from features
By default TreeNodes have a number of features associated with them (support, height, dist, idx, name) and these are often useful for styling nodes. You can also add custom features to nodes (see TreeNodes chapter). Here I set the size and color of nodes based on features of nodes in a random tree (the random node names).

In [4]:
# generate a random tree
ntips = 20
rseed = 12345
rtre = toytree.rtree.coaltree(ntips=ntips, seed=rseed)

# assign new feature 'ancstate' as the mean value of the names of descendant tips
for node in rtre.treenode.traverse():
    value = np.mean([int(i.name[1:]) for i in node.get_leaves()])
    node.add_feature("ancstate", value)
    
# get values in node plot order without tip values shown
sizes = rtre.get_node_values("ancstate", True, False)

# use a boolean of whether 'ancstate' is >10 to set color
colors = [toytree.colors[0] if (i and i>10) else toytree.colors[1] for i in sizes]
    
# draw tree with styles
rtre.draw(
    node_labels=False,
    node_sizes=sizes,
    node_colors=colors,
    node_style={"stroke": "black"}
);

<svg class="toyplot-canvas-Canvas" height="360.0px" id="tfca7085a18fe4d73a9212f9dc2fce72a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 360.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r18 r9 r8 r16 r7 r15 r11 r5 r6 r19 r2 r0 r13 r4 r12 r1 r17 r3 r14 r10

### 4. Variable edge colors and widths  
The function `.get_edge_values_from_dict()` is the most convenient way to apply a style value to parts of the tree. It returns a list with the value (e.g., color or width) mapped to the correct index of the list to apply to the correct edge when entered as an argument to draw. This is convenient for applying styles to clades. If alternatively you want to apply style to individual edges it is best to use `.get_edge_values()` and use the 'idx' argument to return the index order in which edges are plotted. You can then create a list of edge values based on this order. Both examples are shown below, as well as a way of shifting node labels so they are arranged over edges. The 'idx' label of nodes is used to refer to edges subtending nodes.


In [5]:
# generate a random tree
ntips = 7
rseed = 123
rtre = toytree.rtree.coaltree(ntips=ntips, seed=rseed)

# edge mapping 1: enter a dictionary mapping clade members to colors
ecolors = rtre.get_edge_values_from_dict({
    ("r1", "r4", "r5"): toytree.colors[0],  # <- using tips to define a clade
    ("r3", "r0"): toytree.colors[1],        # <- using tips to define a clade 
    12: toytree.colors[2],                  # <- usign node idx to define a clade
})

# edge mapping 2: map specific edges (here 3,6,10,11,12) to edge width value
elabels = rtre.get_edge_values('idx')
ewidths = [3.5 if i in (3, 6, 10, 11, 12) else 2.5 for i in elabels]

# draw tree with edge colors, edge_widths, and node idx labels shifted to edges
c, a = rtre.draw(
    edge_colors=ecolors, 
    edge_widths=ewidths,
    node_sizes=0, 
    node_labels=rtre.get_node_values('idx', True, True),
    node_labels_style={
        "-toyplot-anchor-shift": "-10px",
        "baseline-shift": "5px"
    },
);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t58000056adaf4adfa2dcefbf98391ee6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 275.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r4 r5 r1 r2 r3 r0 r6 0 1 2 3 4 5 6 7 8 9 10 11 12

### 5. Colored rectangles to highlight clades  
The easiest way to add colored shapes to a plot is with the Toyplot `.rectangle` or `.fill()` functions of cartesian `axes` objects. For this you simply need to know the coordinates of the area that you wish to fill (See [Coordinates](https://toytree.readthedocs.io/en/latest/4-tutorial.html#Drawing:-The-Canvas,-Axes,-and-coordinates) for details on this). The example below draws two rectangles in the coordinate space and then adds a tree on top of these. You could make more complex polygon shapes using the `fill` function (see Toyplot docs). 

In [6]:
# generate a random tree
rtre = toytree.rtree.unittree(20, seed=12345)

# make the canvas and axes
canvas = toyplot.Canvas(width=300, height=400)
axes = canvas.cartesian()
axes.show = False

# draw a rectangle (x1, x2, y1, y2)
axes.rectangle(
    -0.75, 0.35, -0.5, 4.5, 
    opacity=0.25,
    color=toytree.colors[0],
)

# draw a rectangle (x1, x2, y1, y2)
axes.rectangle(
    -0.75, 0.35, 4.5, 8.5, 
    opacity=0.25,
    color=toytree.colors[1],
)

# add tree to the axes 
rtre.draw(axes=axes);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="tf9c6924d37874a63a7f7bf3a20a30528" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 400.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r6 r2 r19 r10 r16 r0 r7 r5 r14 r8 r17 r11 r15 r12 r1 r4 r9 r3 r13 r18

### 6. Plot histograms associated with tip trait values (ridge plot)
You can use the `.hist()` or `.fill()` functions of toytree to plot histograms. Here we will generate and plot a distribution of a data in order from top to bottom so that the histograms overlap in a "ridge plot" fashion. The analagous function in `ggtree` apparently merited it's own publication: https://academic.oup.com/mbe/article/35/12/3041/5142656. 

In [212]:
# generate a random tree with N tips
ntips = 40
tre = toytree.rtree.baltree(ntips).mod.node_slider(seed=123)

# generate a distribution between -10 and 10 for each tip in the tree
points = np.linspace(-10, 10, 50)
dists = {}
for tip in tre.get_tip_labels():
    dists[tip] = sc.norm.pdf(points, loc=np.random.randint(-5, 5, 1), scale=2)

In [216]:
# set up canvas for two panel plot
canvas = toyplot.Canvas(width=375, height=400)

# add tree to canvas
ax0 = canvas.cartesian(bounds=(50, 180, 50, 350), ymin=0, ymax=ntips, padding=15)
tre.draw(axes=ax0, tip_labels=False)
ax0.show = False

# add histograms to canvas
ax1 = canvas.cartesian(bounds=(200, 325, 50, 350), ymin=0, ymax=ntips, padding=15)

# iterate from top to bottom (ntips to 0)
for tip in range(tre.ntips)[::-1]:
    
    # select a color for hist
    color = toytree.colors[int((tip) / 10)]
    
    # get tip name and get hist from dict
    tipname = tre.get_tip_labels()[tip]
    probs = dists[tipname]
    
    # fill histogram with slightly overlapping histograms
    ax1.fill(
        points, probs / probs.max() * 1.25,
        baseline=[tip] * len(points), 
        style={"fill": color, "stroke": "white", "stroke-width": 0.5},
        title=tipname,
            )
    
    # add horizontal line at base
    ax1.hlines(tip, opacity=0.5, color="grey", style={"stroke-width": 0.5})

# hide y axis, show x 
ax1.y.show = False
ax1.x.label.text = "Trait size"
ax1.x.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t07bbfcc3d0c1449c8c934403fda9df39" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 375.0 400.0" width="375.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="200.0,57.50000000000001 202.55102040816325,57.50000000000001 205.10204081632654,57.50000000000001 207.65306122448982,57.50000000000001 210.20408163265307,57.50000000000001 212.75510204081633,57.50000000000001 215.30612244897958,57.50000000000001 217.85714285714286,57.50000000000001 220.40816326530614,57.50000000000001 222.95918367346937,57.50000000000001 225.51020408163265,57.50000000000001 228.06122448979593,57.50000000000001 230.61224489795916,57.50000000000001 233.16326530612247,57.50000000000001 235.71428571428572,57.50000000000001 238.26530612244898,57.50000000000001 240.81632653061223,57.50000000000001 243.3673469387755,57.50000000000001 245.9183673469388,57.50000000000001 248.46938775510205,57.50000000000001 251.0204081632653,57.50000000000001 253.57142857142856,57.50000000000001 256.1224489795918,57.50000000000001 258.6734693877551,57.50000000000001 261.2244897959184,57.50000000000001 263.7755102040816,57.50000000000001 266.3265306122449,57.50000000000001 268.8775510204082,57.50000000000001 271.42857142857144,57.50000000000001 273.9795918367347,57.50000000000001 276.53061224489795,57.50000000000001 279.0816326530612,57.50000000000001 281.6326530612245,57.50000000000001 284.18367346938777,57.50000000000001 286.734693877551,57.50000000000001 289.2857142857143,57.50000000000001 291.8367346938776,57.50000000000001 294.38775510204084,57.50000000000001 296.9387755102041,57.50000000000001 299.48979591836735,57.50000000000001 302.0408163265306,57.50000000000001 304.59183673469386,57.50000000000001 307.1428571428571,57.50000000000001 309.6938775510204,57.50000000000001 312.2448979591837,57.50000000000001 314.795918367347,57.50000000000001 317.34693877551024,57.50000000000001 319.89795918367344,57.50000000000001 322.4489795918367,57.50000000000001 325.0,57.50000000000001 325.0,57.49999999999431 322.4489795918367,57.499999999974094 319.89795918367344,57.499999999887464 317.34693877551024,57.49999999953135 314.795918367347,57.499999998128274 312.2448979591837,57.499999992828876 309.6938775510204,57.499999973646524 307.1428571428571,57.49999990710302 304.59183673469386,57.4999996858929 302.0408163265306,57.49999898125475 299.48979591836735,57.499996830685 296.9387755102041,57.49999054248234 294.38775510204084,57.49997292921536 291.8367346938776,57.49992567473576 289.2857142857143,57.4998042580038 286.734693877551,57.49950552595819 284.18367346938777,57.49880183972814 281.6326530612245,57.49721517270887 279.0816326530612,57.49379140185671 276.53061224489795,57.486722975144396 273.9795918367347,57.472765467846465 271.42857142857144,57.446414041507566 268.8775510204082,57.39886673674415 266.3265306122449,57.316916598938114 263.7755102040816,57.182081459839196 261.2244897959184,56.97046499701879 258.6734693877551,56.65397066205449 256.1224489795918,56.203453469713416 253.57142857142856,55.594088526789506 251.0204081632653,54.81261879881557 248.46938775510205,53.86530679929338 245.9183673469388,52.78460394790178 243.3673469387755,51.632129430636276 240.81632653061223,50.495860143232484 238.26530612244898,49.48061569559044 235.71428571428572,48.69277327976977 233.16326530612247,48.22210913340732 230.61224489795916,48.12499999999998 228.06122448979593,48.41332016068802 225.51020408163265,49.05205438022004 222.95918367346937,49.96628952260367 220.40816326530614,51.05565658065093 217.85714285714286,52.21237850945152 215.30612244897958,53.3384609386508 212.75510204081633,54.35833752258875